## 1. Import Libraries and Load Data

In [ ]:
# Libraries
import warnings
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.neighbors import NearestNeighbors
from sklearn.manifold import TSNE, trustworthiness
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer, InterclusterDistance

# Set-up environment
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_colwidth', None)
sns.set_theme(style="whitegrid", context="paper")
os.chdir('/Users/nataschajademinnitt/Documents/5. Data Analysis/segmenting_customers/')
print("Current directory:", os.getcwd())
warnings.filterwarnings("ignore")

In [ ]:
# Load the data
df = pd.read_csv("./data/processed/processed_database.csv")
df.info()

## 2. Functions

In [ ]:
def evaluate_feature_sets(
    df,
    dict_features,
    scaler,
    k_min: int = 2,
    k_max: int = 10,
    random_state: int = 42
) -> pd.DataFrame:
    """
    Uses a dictionary of features to find the best k elbow.
    Computes silhouette, calinski_harabasz and davies_bouldin at that k.
    """
    results = []

    for name, cols in dict_features.items():
        X = df[cols].values
        
        # use the passed‑in scaler
        X_scaled = scaler.fit_transform(X)
        
        # find the elbow
        model = KMeans(random_state=random_state)
        viz = KElbowVisualizer(
            model,
            k=(k_min, k_max),
            metric='distortion',
            show=False,
            timings=False
        )
        viz.fit(X_scaled)
        best_k = viz.elbow_value_
        
        # compute internal scores
        labels = KMeans(n_clusters=best_k, random_state=random_state).fit_predict(X_scaled)
        sil = silhouette_score(X_scaled, labels)
        ch  = calinski_harabasz_score(X_scaled, labels)
        db  = davies_bouldin_score(X_scaled, labels)
        
        results.append({
            'feature_set': name,
            'scaler': scaler.__class__.__name__,
            'best_k_elbow': best_k,
            'silhouette_score': sil,
            'calinski_harabasz_score': ch,
            'davies_bouldin_score': db
        })

    return pd.DataFrame(results)

In [ ]:
def evaluate_clusters(
    data,
    feature_cols=None,
    metric="distortion",
    k_min=2,
    k_max=10,
    random_state=42
):
    """
    Evaluate clustering (elbow, silhouette, intercluster distance) for a set of features.
    
    """

    # 2) Subset & scale once
    X = data[feature_cols].values
    scaler = RobustScaler().fit(X)
    X_scaled = scaler.transform(X)

    # 3) Make your 3‑panel canvas
    fig, axes = plt.subplots(3, 1, figsize=(6, 10))

    # Elbow
    kelbow = KElbowVisualizer(
        KMeans(random_state=random_state),
        k=(k_min, k_max),
        metric=metric,
        ax=axes[0]
    )
    kelbow.fit(X_scaled)
    best_k = kelbow.elbow_value_
    kelbow.finalize()

    # Silhouette
    silvis = SilhouetteVisualizer(
        KMeans(n_clusters=best_k, random_state=random_state),
        ax=axes[1]
    )
    silvis.fit(X_scaled)
    silvis.finalize()

    # Inter‑cluster distances
    icd = InterclusterDistance(
        KMeans(n_clusters=best_k, random_state=random_state),
        ax=axes[2]
    )
    icd.fit(X_scaled)
    icd.finalize()

    plt.tight_layout()
    plt.show()

    return best_k, fig

In [ ]:
def plot_cluster_radar(df, title="Cluster Radar Plot"):
    """
    Create a radar (spider) plot for cluster profiles.
    """
    # Number of variables
    features = df.columns.tolist()
    num_vars = len(features)

    # Compute angles for each axis
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]  # complete the loop

    # Setup polar plot
    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    fig.suptitle(title, fontsize=14, fontweight='bold')

    # Plot each cluster
    for idx, row in df.iterrows():
        values = row.values.flatten().tolist()
        values += values[:1]
        ax.plot(angles, values, linewidth=2, label=str(idx))
        ax.fill(angles, values, alpha=0.25)

    # Add feature labels on the axes
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(features)

    # Add legend
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))

    plt.tight_layout()
    plt.show()

## K-Means segmentation

In [ ]:
# Data prep & scaling
features = ['recency','f_returning','m_price_log','s_review_score']
X = df[features].values
X_scaled = RobustScaler().fit_transform(X)

In [ ]:
for k in range(3, 6):
    labels = KMeans(n_clusters=k, random_state=42).fit_predict(X_scaled)
    sil = silhouette_score(X_scaled, labels)
    ch  = calinski_harabasz_score(X_scaled, labels)
    db  = davies_bouldin_score(X_scaled, labels)
    print(f"k={k}: silhouette={sil:.3f}, CH={ch:.1f}, DB={db:.3f}")

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(8, 12))

# 3. Elbow visualizer
kelbow = KElbowVisualizer(
    KMeans(random_state=42),
    k=(2,10),
    metric='distortion',
    ax=axes[0]
)
kelbow.fit(X_scaled)
kelbow.finalize()

# 4. Silhouette visualizer (using best_k from elbow)
best_k = kelbow.elbow_value_
silvis = SilhouetteVisualizer(
    KMeans(n_clusters=best_k, random_state=42),
    ax=axes[1]
)
silvis.fit(X_scaled)
silvis.finalize()

plt.tight_layout()
plt.show()

In [ ]:
km = KMeans(n_clusters=4, random_state=42).fit(X_scaled)
df['cluster'] = km.labels_

In [ ]:
counts = df['cluster'].value_counts().sort_index()
print(pd.DataFrame({'cluster':counts.index, 'count':counts.values}))

In [ ]:
Y = TSNE(
    perplexity=50,
    init='pca',
    random_state=42
).fit_transform(X_scaled)

In [ ]:
# Assume Y is your t-SNE embedding of the same samples
fig = px.scatter(
    x=Y[:,0], y=Y[:,1],
    color=df['cluster'].astype(str),
    title="t-SNE of 4 Clusters",
    labels={'color':'Cluster'}
)
fig.write_image(f"plots/tsne_kmeans.png", width=800, height=600)
fig.show()

In [ ]:
# Plot results for
os.makedirs("plots", exist_ok=True)

for col in features:
    fig = px.scatter(
        x=Y[:, 0],
        y=Y[:, 1],
        color=df[col],
        color_continuous_scale='Viridis',
        title=f"t-SNE of Customers colored by {col}",
        labels={'x': 't-SNE 1', 'y': 't-SNE 2', 'color': col}
    )
    fig.write_image(f"plots/tsne_{col}.png", width=800, height=600)
    fig.show()

In [ ]:
for feature in ['recency', 'f_returning', 'm_price', 's_review_score']:
    fig = px.violin(
        df, x='cluster', y=feature, points=False,
        title=f"Distribution of {feature} by cluster"
    )
    fig.write_image(f"plots/violin_{feature}.png", width=800, height=600)
    fig.show()

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

# --- 1. Prepare your data split into time windows ---
# Suppose df has a Date column and your features+scale come from df
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')

# Define your windows, e.g. monthly
start = df['date'].min()
windows = []
for i in range(6):   # say you want 6 months of monitoring
    t0 = start + pd.DateOffset(months=i)
    t1 = t0 + pd.DateOffset(months=1)
    windows.append(df[(df['date'] >= t0) & (df['date'] < t1)])
    
# --- 2. Define your candidate feature subsets ---
candidates = [
    ['recency','f_returning','m_price_log','s_review_score'],   # rfms
    ['recency','f_returning','m_price_log','s_review_score','s_delivery_diff_binary'],  # rfmss
    # add any other combos you want to test
]

random_state = 42
K = 4    # fixed number of clusters

results = []

for feat_set in candidates:
    # 3A. Baseline clustering on window 0
    X0 = windows[0][feat_set].values
    X0s = RobustScaler().fit_transform(X0)
    labels0 = KMeans(n_clusters=K, random_state=random_state).fit_predict(X0s)
    
    # 3B. Step through future windows until ARI < 0.8
    survival = 0
    for w in windows[1:]:
        Xi = w[feat_set].values
        if len(Xi) < K: 
            break   # not enough points to cluster
        Xi_s = RobustScaler().fit_transform(Xi)  # or reuse scaler? up to you
        li = KMeans(n_clusters=K, random_state=random_state).fit_predict(Xi_s)
        ari = adjusted_rand_score(labels0, li)
        if ari >= 0.8:
            survival += 1
        else:
            break
    
    results.append({
        'features': feat_set,
        'survival_months': survival
    })

# 4. Rank & display
res_df = pd.DataFrame(results).sort_values('survival_months', ascending=False)
print(res_df.to_string(index=False))

## 3. Comparing Models

### 3.1. K-Means Clustering

#### Baseline Performance

Comaring the baseline performance of k-means clustering for feature combinations using different sclaers: MinMax, Standard, Robust.

In [ ]:
dict_features = {
    # 1) RFM baseline
    'rfm': ['recency', 'f_returning', 'm_price_log'],

    # 2) RFM + s_review_score
    'rfm_score': ['recency', 'f_returning', 'm_price_log', 's_review_score'],

    # 3) RFM + s_delivery_diff_binary
    'rfm_deliv': ['recency', 'f_returning', 'm_price_log', 's_delivery_diff_binary'],
    
    # 4) RFM + m_pct_freight
    'rfm_freight': ['recency', 'f_returning', 'm_price_log', 'm_pct_freight'],

    # 5) RFM + m_credit
    'rfm_credit': ['recency', 'f_returning', 'm_price_log', 'f_basket_size']
    
}

In [ ]:
# Compare clustering performance
res_mm = evaluate_feature_sets(df, dict_features, scaler=MinMaxScaler())
res_std = evaluate_feature_sets(df, dict_features, scaler=StandardScaler())
res_rob = evaluate_feature_sets(df, dict_features, scaler=RobustScaler())

In [ ]:
# Display results df
df_baseline = pd.concat([res_mm, res_std, res_rob], ignore_index=True)
df_baseline.sort_values('silhouette_score', ascending=False)

#### Recency and Monetary adjustments

Comparing recency as a continious and categorical feature with monetary as continious and ratio of installments.

In [ ]:
df.info()

In [ ]:
dict_features = {
    'rfm_cont_instal': ['recency', 'f_returning', 'm_value_installments', 'f_basket_size', 's_delivery_diff_binary'],

    'rfm_cont_log': ['recency', 'f_returning', 'm_price_log', 'f_basket_size', 's_delivery_diff_binary'],
    
    'rfm_flag_instal': ['recent_flag', 'f_returning', 'm_value_installments', 'f_basket_size', 's_delivery_diff_binary'],
    
    'rfm_flag_log': ['recent_flag', 'f_returning', 'm_price_log', 'f_basket_size', 's_delivery_diff_binary'],
}

In [ ]:
# Compare clustering performance
results_df = evaluate_feature_sets(df, dict_features, scaler=MinMaxScaler())
results_df

#### RFMS and other features

m_value_installments doesn't seem to clearly differentiate clusters so it's been replaced by m_price_log.

Below additional features are added to create more nuanced categories.

In [ ]:
dict_features = {
    'rfm_1': ['recent_flag', 'f_returning', 'm_price_log', 'f_basket_size', 's_delivery_diff_binary', 'm_purchasing_power'],

    'rfm_2': ['recent_flag', 'f_returning', 'm_price_log', 'f_basket_size', 's_delivery_diff_binary', 'm_total_installments'],
    
    'rfm_3': ['recent_flag', 'f_returning', 'm_price_log', 'f_basket_size', 's_delivery_diff_binary', 'm_credit'],
    
    'rfm_4': ['recent_flag', 'f_returning', 'm_price_log', 'f_basket_size', 's_delivery_diff_binary', 'state_spending'],
    
    'rfm_5': ['recent_flag', 'f_returning', 'm_price_log', 'f_basket_size', 's_delivery_diff_binary', 'm_pct_freight'],
}

In [ ]:
# Compare clustering performance
results_df = evaluate_feature_sets(df, dict_features, scaler=MinMaxScaler())
results_df

In [ ]:
# Radar plot
features = ['recent_flag', 'f_returning', 'm_price_log', 'f_basket_size', 's_delivery_diff_binary', 'm_total_installments']

X = df[features].values
X_scaled = MinMaxScaler().fit_transform(X)

kmeans = KMeans(n_clusters=4, random_state=42).fit(X_scaled)
labels = kmeans.labels_

dfb = pd.DataFrame(X_scaled, columns=features)
dfb['cluster'] = labels

profile = dfb.groupby('cluster').mean().round(2)

plot_cluster_radar(profile)
profile

In [ ]:
# Violin plot
fig, axes = plt.subplots(1, len(features), figsize=(len(features)*4, 6), sharey=False)

for ax, feature in zip(axes, features):
    sns.violinplot(
        x='cluster', y=feature,
        data=dfb, ax=ax,
        inner='quartile', scale='width'
    )
    ax.set_title(feature)
    ax.set_xlabel('Cluster')
    ax.set_ylabel(feature)

fig.tight_layout()
plt.show()

The additional features seem to add noise rather than nuance. We will revert back to the simplified RFMS model. 

### 3.2. DBSCAN

In [ ]:
X_cont = RobustScaler().fit_transform(df[['m_price_log','f_basket_size']])
X_mm   = MinMaxScaler().fit_transform(np.hstack([df[['recent_flag','f_returning','s_delivery_diff_binary']].values, X_cont]))

In [ ]:
rfm_install_orders = ['recent_flag', 'f_returning', 'm_price_log', 'f_basket_size', 's_delivery_diff_binary']
X = df[rfm_install_orders].values
X_scaled = MinMaxScaler().fit_transform(X)

In [ ]:
nbrs = NearestNeighbors(n_neighbors=5).fit(X_scaled)
dists, _ = nbrs.kneighbors(X_scaled)
dists = np.sort(dists[:,4])
plt.plot(dists)
plt.ylabel("5th nearest neighbor distance")
plt.xlabel("Points sorted by distance")
plt.ylim(0, 0.2)
plt.axhline(0.5, linestyle='--', color='red')
plt.savefig(f"./results/5th_nearest_neighbor_distance.png")
plt.show()

In [ ]:
# DBSCAN clustering
eps_values        = [0.01, 0.05, 0.15, 0.20]
min_samples_vals  = [5, 10, 50]

results = []
for eps in eps_values:
    for ms in min_samples_vals:
        db = DBSCAN(eps=eps, min_samples=ms).fit(X_scaled)
        labels = db.labels_
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise    = (labels == -1).sum()
        sil = silhouette_score(X_scaled[labels != -1], labels[labels != -1])

        results.append({
            'eps':            eps,
            'min_samples':    ms,
            'n_clusters':     n_clusters,
            'n_noise':        n_noise,
            'silhouette':     sil
        })

df_results = pd.DataFrame(results)
df_results.sort_values(['silhouette','n_clusters'], ascending=[False,True])

In [ ]:
# DBSCAN clustering
db = DBSCAN(eps=0.2, min_samples=50).fit(X_scaled)
labels_db = db.labels_
n_db = len(set(labels_db)) - (1 if -1 in labels_db else 0)
print(f"DBSCAN found {n_db} clusters (and {(labels_db==-1).sum()} noise points)")

In [ ]:
# Silhouette score for DBSCAN
print("Silhouette:", silhouette_score(X_scaled[labels_db!=-1], labels_db[labels_db!=-1]))

In [ ]:
# Observations per cluster
unique, counts = np.unique(labels_db, return_counts=True)
print(dict(zip(unique, counts)))

In [ ]:
# Radar plot
df_db = pd.DataFrame(X_scaled, columns=rfm_install_orders)
df_db['cluster'] = labels_db
profile = df_db[df_db['cluster'] != -1].groupby('cluster').mean()
plot_cluster_radar(profile)
profile.round(3)

## 4. Best combination of features

In [ ]:
# Radar plot
features = ['recent_flag', 'f_returning', 'm_price_log', 's_delivery_diff_binary']

X = df[features].values
X_scaled = MinMaxScaler().fit_transform(X)

kmeans = KMeans(n_clusters=4, random_state=42).fit(X_scaled)
labels = kmeans.labels_

dfb = pd.DataFrame(X_scaled, columns=features)
dfb['cluster'] = labels

profile = dfb.groupby('cluster').mean().round(2)

plot_cluster_radar(profile)
profile

In [ ]:
# Violin plot
fig, axes = plt.subplots(1, len(features), figsize=(len(features)*4, 6), sharey=False)

for ax, feature in zip(axes, features):
    sns.violinplot(
        x='cluster', y=feature,
        data=dfb, ax=ax,
        inner='quartile', scale='width'
    )
    ax.set_title(feature)
    ax.set_xlabel('Cluster')
    ax.set_ylabel(feature)

fig.tight_layout()
plt.show()

In [ ]:
# Evaluate clusters
best_k, fig = evaluate_clusters(
    data=df,
    feature_cols=['recent_flag', 'f_returning', 'm_price_log', 's_delivery_diff_binary'],
    metric="distortion",
    k_min=2,
    k_max=10
)

print("Optimal number of clusters according to the elbow for RFMR:", best_k)